In [4]:
!python --version

Python 3.10.12


In [5]:
import gc
import tensorflow as tf

# Custom Callbac
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

# Installing required Libraries

In [6]:
!pip install tabula-py
!pip install pypdf
!pip install PyPDF2
!pip install fuzzywuzzy
!pip install frontend
!pip install wget
!pip install -q transformers einops accelerate langchain bitsandbytes sentence_transformers llama-index pypdf python-dotenv


# Importing required Libraries

In [7]:

import requests
from bs4 import BeautifulSoup
from lxml import etree
from fuzzywuzzy import fuzz
import PyPDF2
import wget
from google.colab import files
import tabula
import numpy as np
from IPython.display import display




/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Functions to parse the DOM

In [8]:
def get_dom_object(url,HEADERS):

  response = requests.get(url,HEADERS)
  #print(url,response.status_code )
  soup = BeautifulSoup(response.content, "html.parser")
  dom = etree.HTML(str(soup))

  return dom,(response.status_code == 200)

In [9]:
def get_data_with_xpath(dom,xpath,company_code,base_url):

  part=None
  report_url=""
  if(company_code=="MGS"):
    element_list=dom.xpath(xpath)
    if(element_list):
      element=element_list[0]
      part=element.attrib["href"]
      report_url=base_url+part
      #print(report_url)
      return report_url,(part!=None)


In [10]:
def get_company_data(dom,comany,current_year):

  data={}
  if(comany["Code"]=="MGS"):
    for year in range(comany["report_avalaible_from"],comany["report_avalaible_till"]-1,-1):
        data[str(year)]=[]
        for quater in range(2,6):
          obj={}
          obj["Quarter"]=quater-1
          obj["FinancialSuppliments"]=""
          obj["isAvailabe"]=False
          index=current_year-year
          xpath_morgan_stanley=f"//html//body//div[1]//div[2]//div//div//div//section//article//table//tbody//tr[{index}]//td[{quater}]//a[2]"
          #print(xpath_morgan_stanley)

          report_url,is_available=get_data_with_xpath(dom,xpath_morgan_stanley,comany["Code"],comany["base_url"])
          if(is_available):
            obj["Financial Suppliments"]=report_url
            obj["isAvailabe"]=True
          data[str(year)].append(obj)
    return data


# Companies List to retreive data

In [11]:
company_list=[

             {   "Name":"Morgan Stanley",
                 "Code":"MGS",
                 "report_avalaible_from":2023,
                 "report_avalaible_till":2002,
                 "url":"https://www.morganstanley.com/about-us-ir/earnings-releases",
                 "base_url":"https://www.morganstanley.com"
             }

]
current_year=2024

HEADERS = ({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 \ (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})


In [12]:

all_comany_pdfs=[]
data=[]
for comany in company_list:
  dom,is_success=get_dom_object(comany["url"],HEADERS)
  if(is_success):
    data=get_company_data(dom,comany,current_year)

  all_comany_pdfs.append({comany["Code"] : data })





In [13]:
all_comany_pdfs[0]['MGS']

{'2023': [{'Quarter': 1,
   'FinancialSuppliments': '',
   'isAvailabe': True,
   'Financial Suppliments': 'https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup1q2023/finsup1q2023.pdf'},
  {'Quarter': 2,
   'FinancialSuppliments': '',
   'isAvailabe': True,
   'Financial Suppliments': 'https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup2q2023/finsup2q2023.pdf'},
  {'Quarter': 3,
   'FinancialSuppliments': '',
   'isAvailabe': True,
   'Financial Suppliments': 'https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup3q2023/finsup3q2023.pdf'},
  {'Quarter': 4,
   'FinancialSuppliments': '',
   'isAvailabe': True,
   'Financial Suppliments': 'https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup4q2023/finsup4q2023.pdf'}],
 '2022': [{'Quarter': 1,
   'FinancialSuppliments': '',
   'isAvailabe': True,
   'Financial Suppliments': 'https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup1q2022/finsup1q

In [14]:
all_comany_pdfs[0]['MGS']['2023']

[{'Quarter': 1,
  'FinancialSuppliments': '',
  'isAvailabe': True,
  'Financial Suppliments': 'https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup1q2023/finsup1q2023.pdf'},
 {'Quarter': 2,
  'FinancialSuppliments': '',
  'isAvailabe': True,
  'Financial Suppliments': 'https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup2q2023/finsup2q2023.pdf'},
 {'Quarter': 3,
  'FinancialSuppliments': '',
  'isAvailabe': True,
  'Financial Suppliments': 'https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup3q2023/finsup3q2023.pdf'},
 {'Quarter': 4,
  'FinancialSuppliments': '',
  'isAvailabe': True,
  'Financial Suppliments': 'https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup4q2023/finsup4q2023.pdf'}]

#Extract Pages that has Wealth Management

In [15]:

def extract_pages_with_text(pdf_path, search_texts, threshold=80):
    matching_pages = []
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
          page = pdf_reader.pages[page_num]
          page_text = page.extract_text()

          similarity = 0
          for search_text in search_texts:
            similarity+=fuzz.partial_ratio(search_text.lower(), page_text.lower())

          if similarity >= threshold*len(search_texts):
              matching_pages.append(page_num + 1)

    return matching_pages

In [16]:
pdf_url = 'https://www.morganstanley.com/content/dam/msdotcom/en/about-us-ir/finsup1q2023/finsup1q2023.pdf'

local_path = "/content/finsup1q2023.pdf"

wget.download(pdf_url, local_path)

search_texts = ['Wealth Management','Income Statement Information']

matching_pages = extract_pages_with_text(local_path, search_texts)

if matching_pages:
    print(f"Pages with '{search_texts}': {matching_pages}")
else:
    print(f"No pages found with '{search_texts}'")


Pages with '['Wealth Management', 'Income Statement Information']': [7, 8]


#Extract Required Tables from Pdf

In [17]:
def remove_unnecesary_cols(tables,threshold):
  tables_new=[]
  for i in range(len(tables)):
    data_frame=tables[i]
    data_frame=data_frame.dropna(thresh=len(data_frame)-threshold, axis=1)
    tables_new.append(data_frame)
  return tables_new



In [18]:
tables = tabula.read_pdf(pdf_url, pages= matching_pages, multiple_tables=True)
tables[1]

,Unnamed: 0,Unnamed: 1,"Mar 31, 2023",Unnamed: 2,"Dec 31, 2022",Unnamed: 3,"Mar 31, 2022","Dec 31, 2022.1","Mar 31, 2022.1"
0,ealth Management Metrics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total client assets,$,"4,558",$,"4,187",$,"4,869",9%,(6%)
2,Net new assets,$,109.6,$,51.6,$,142.0,112%,(23%)
3,U.S. Bank loans,$,143.7,$,146.1,$,136.7,(2%),5%
4,Margin and other lending (1),$,21.1,$,22.0,$,29.2,(4%),(28%)
5,Deposits (2),$,341,$,351,$,352,(3%),(3%)
6,Annualized weighted average cost of deposits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Period end,NaN,2.05%,NaN,1.59%,NaN,0.09%,NaN,NaN
8,Period average,NaN,1.86%,NaN,1.32%,NaN,0.10%,NaN,NaN
9,dvisor-led channel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:

tables=remove_unnecesary_cols(tables,threshold=10)

tables[1]

,Unnamed: 0,"Mar 31, 2023","Dec 31, 2022","Mar 31, 2022","Dec 31, 2022.1","Mar 31, 2022.1"
0,ealth Management Metrics,NaN,NaN,NaN,NaN,NaN
1,Total client assets,"4,558","4,187","4,869",9%,(6%)
2,Net new assets,109.6,51.6,142.0,112%,(23%)
3,U.S. Bank loans,143.7,146.1,136.7,(2%),5%
4,Margin and other lending (1),21.1,22.0,29.2,(4%),(28%)
5,Deposits (2),341,351,352,(3%),(3%)
6,Annualized weighted average cost of deposits,NaN,NaN,NaN,NaN,NaN
7,Period end,2.05%,1.59%,0.09%,NaN,NaN
8,Period average,1.86%,1.32%,0.10%,NaN,NaN
9,dvisor-led channel,NaN,NaN,NaN,NaN,NaN


#Split dataframes from tables

In [20]:
def split_data_frame(data_frame,nan_threshold):
  nan_rows = data_frame[data_frame.isna().sum(axis=1) >= nan_threshold]
  column_to_extract='Unnamed: 0'
  column_data = nan_rows[column_to_extract]
  split_dataframes = np.split(data_frame, nan_rows.index)
  return split_dataframes,column_data


In [21]:
splited_dataframes,column_data=split_data_frame(tables[1],tables[1].shape[1]-2)

In [22]:
len(column_data)
column_data=list(column_data)

In [23]:
len(splited_dataframes)

6

In [24]:
for i, df in enumerate(splited_dataframes):

    display(df)

,Unnamed: 0,"Mar 31, 2023","Dec 31, 2022","Mar 31, 2022","Dec 31, 2022.1","Mar 31, 2022.1"


,Unnamed: 0,"Mar 31, 2023","Dec 31, 2022","Mar 31, 2022","Dec 31, 2022.1","Mar 31, 2022.1"
0,ealth Management Metrics,NaN,NaN,NaN,NaN,NaN
1,Total client assets,"4,558","4,187","4,869",9%,(6%)
2,Net new assets,109.6,51.6,142.0,112%,(23%)
3,U.S. Bank loans,143.7,146.1,136.7,(2%),5%
4,Margin and other lending (1),21.1,22.0,29.2,(4%),(28%)
5,Deposits (2),341,351,352,(3%),(3%)


,Unnamed: 0,"Mar 31, 2023","Dec 31, 2022","Mar 31, 2022","Dec 31, 2022.1","Mar 31, 2022.1"
6,Annualized weighted average cost of deposits,NaN,NaN,NaN,NaN,NaN
7,Period end,2.05%,1.59%,0.09%,NaN,NaN
8,Period average,1.86%,1.32%,0.10%,NaN,NaN


,Unnamed: 0,"Mar 31, 2023","Dec 31, 2022","Mar 31, 2022","Dec 31, 2022.1","Mar 31, 2022.1"
9,dvisor-led channel,NaN,NaN,NaN,NaN,NaN
10,Advisor-led client assets,"3,582","3,392","3,835",6%,(7%)
11,Fee-based client assets,"1,769","1,678","1,873",5%,(6%)
12,Fee-based asset flows,22.4,20.4,97.2,10%,(77%)
13,Fee-based assets as a % of advisor-led client ...,49%,49%,49%,NaN,NaN


,Unnamed: 0,"Mar 31, 2023","Dec 31, 2022","Mar 31, 2022","Dec 31, 2022.1","Mar 31, 2022.1"
14,elf-directed channel,NaN,NaN,NaN,NaN,NaN
15,Self-directed assets,976,795,"1,034",23%,(6%)
16,Daily average revenue trades (000's),8 31,7 55,"1,016",10%,(18%)
17,Self-directed households (millions),8.1,8.0,7.6,1%,7%


,Unnamed: 0,"Mar 31, 2023","Dec 31, 2022","Mar 31, 2022","Dec 31, 2022.1","Mar 31, 2022.1"
18,orkplace channel,NaN,NaN,NaN,NaN,NaN
19,Stock plan unvested assets,358,302,454,19%,(21%)
20,Number of stock plan participants (millions),6.5,6.3,5.8,3%,12%


In [25]:
data_to_llm=[]

for i, df in enumerate(splited_dataframes):
  if(i==0):
    continue
  data_str=""

  df=df.iloc[1:]
  print(f"Table  {column_data[i-1]} ")
  data_str+=f"Table  {column_data[i-1]} "

  data_str+="\n"

  print()
  df_array=remove_unnecesary_cols([df],1)

  print(df_array[0].to_markdown())
  data_str+=df_array[0].to_markdown()
  data_str+="\n\n"
  print()
  print()
  print()
  data_to_llm.append(data_str)


Table  ealth Management Metrics 

|    | Unnamed: 0                   | Mar 31, 2023   | Dec 31, 2022   | Mar 31, 2022   | Dec 31, 2022.1   | Mar 31, 2022.1   |
|---:|:-----------------------------|:---------------|:---------------|:---------------|:-----------------|:-----------------|
|  1 | Total client assets          | 4,558          | 4,187          | 4,869          | 9%               | (6%)             |
|  2 | Net new assets               | 109.6          | 51.6           | 142.0          | 112%             | (23%)            |
|  3 | U.S. Bank loans              | 143.7          | 146.1          | 136.7          | (2%)             | 5%               |
|  4 | Margin and other lending (1) | 21.1           | 22.0           | 29.2           | (4%)             | (28%)            |
|  5 | Deposits (2)                 | 341            | 351            | 352            | (3%)             | (3%)             |



Table  Annualized weighted average cost of deposits 

|    | Unnamed: 0   

#Use LLM TO extract any data from the tables


In [26]:
import logging
import sys
import torch
from pprint import pprint
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.readers import StringIterableReader
from llama_index.llms import HuggingFaceLLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
from llama_index import ServiceContext
from llama_index.prompts.prompts import SimpleInputPrompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [27]:
!git config --global credential.helper store
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.ca

In [28]:
# hyperparameters
context_window = 4096 #@param
temperature = 0.0 #@param
model_name = 'meta-llama/Llama-2-13b-chat-hf' #@param

llm = HuggingFaceLLM(
    context_window=context_window,
    max_new_tokens=256,
    generate_kwargs={"temperature": temperature, "do_sample": False},
    tokenizer_name=model_name,
    model_name=model_name,
    device_map="auto",
   #uncomment this if using CUDA to reduce memory usage
    #model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True, "use_auth_token":True},

)
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the disk and cpu.


In [29]:
print(data_to_llm[2])

Table  dvisor-led channel 
|    | Unnamed: 0                                           | Mar 31, 2023   | Dec 31, 2022   | Mar 31, 2022   | Dec 31, 2022.1   | Mar 31, 2022.1   |
|---:|:-----------------------------------------------------|:---------------|:---------------|:---------------|:-----------------|:-----------------|
| 10 | Advisor-led client assets                            | 3,582          | 3,392          | 3,835          | 6%               | (7%)             |
| 11 | Fee-based client assets                              | 1,769          | 1,678          | 1,873          | 5%               | (6%)             |
| 12 | Fee-based asset flows                                | 22.4           | 20.4           | 97.2           | 10%              | (77%)            |
| 13 | Fee-based assets as a % of advisor-led client assets | 49%            | 49%            | 49%            | nan              | nan              |




In [30]:
documents = StringIterableReader().load_data(
    texts=data_to_llm
)

In [31]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)


In [36]:
user_queries = ['what is the value of fee based client assets during march 31, 2023 ?' , 'Self directed assets in Mar 31, 2023 ' ]
answer = dict()
for i, user_query in enumerate(user_queries):
    query_engine = index.as_query_engine()
    torch.cuda.empty_cache()
    response = query_engine.query(user_query)
    answer.update({user_query: response.response})
pprint(answer)

OutOfMemoryError: CUDA out of memory. Tried to allocate 100.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 99.06 MiB is free. Process 33507 has 14.65 GiB memory in use. Of the allocated memory 14.22 GiB is allocated by PyTorch, and 315.15 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
answer

In [35]:
import torch

